## Scratch work

In [3]:
import sys
paths = ['../scripts',
        '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/quickpipeline',
         '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/pd_feature_union' 
        ]

for path in paths:
    sys.path.append(path)
    
from market_imports import *
from market_code import *
from market_trans import *
from quickpipe_mod import * 
from pandas_feature_union import *

data_path = '/Users/jacob/Desktop/docs/kaggle/two_sigma/_g_data/data'
df_market = pd.read_csv(data_path + '/marketdata_sample.csv') 
df_news = pd.read_csv(data_path + '/news_sample.csv')


import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# TRYING TO PREVENT DATE TYPE CONVERSION
# THIS SHOULD BE UPDATED FROM PANDA DATAREADER
csv = pd.read_csv(data_path + '/sigma_data.csv', index_col='Date', parse_dates=True, 
                 infer_datetime_format=True)

csv.shape

(57384, 9)

In [3]:
# COMPUTE RESIDUAL FOR RESPONSE
import statsmodels.tsa.api as tsa
from statsmodels.tsa.seasonal import seasonal_decompose 

x = csv[['returns_open_raw10_next']]
# Freq will depend on asset number - this needs to be researched
result = seasonal_decompose(x, model='additive', freq=50)

csv['returns_res10_next'] = result.resid
# Lose the first and last day
csv = csv[-csv['returns_res10_next'].isnull()]

check = csv.columns[:4].tolist()
csv.drop_duplicates(check, inplace=True)

# ELC is insanely high. The next highest price is around 300
csv[csv.Close ==csv.Close.max()]
csv[csv.Close > 1000]
csv = csv[csv.asset != 'ELC']
csv.shape

(35818, 10)

# class
***

In [30]:
data = csv.copy()

In [33]:
def pipe(original):
    class PipeInto(object):
        data = {'function': original}

        def __init__(self, *args, **kwargs):
            self.data['args'] = args
            self.data['kwargs'] = kwargs

        def __rrshift__(self, other):
            return self.data['function'](
                other, 
                *self.data['args'], 
                **self.data['kwargs']
            )

    return PipeInto


In [36]:

@pipe
def exTractTime(X, col='index', atts=list, mthds=None):

    mod = getattr(X, col)

    for i in range(len(atts)):
    
        X[atts[i]] = getattr(mod, atts[i])
    
    while mthds:
        for i in range(len(mthds)):
    
            f = getattr(mod, mthds[i])
            X[mthds[i]] = f()
        break
        
    return X

    
# exTractTime(data, col='index', 
#                 atts=['weekday', 'month', 'year', 'day', 'quarter'],
#                 mthds=['month_name']).head()    

In [37]:
@pipe
def toCatFeat(X, feats=list):
    for i in feats:
        colName = i + '_obj'
        X[colName] = X[i].astype(str) + '_'
    return X

# toCatFeat(data, feats=['year', 'quarter']).head()   

In [136]:
data = csv.copy()


test =  data >> \
        exTractTime(atts=['weekday', 'month', 'year', 'day', 'quarter'], 
                    mthds=['month_name']) >> \
        toCatFeat(feats=['year', 'quarter']) >> \
        macdFeats() >> \
        bbSqueeze() >> \
        rsiFeats() >> \
        preProcessor()


test.head()

Transforming....


,asset,returns_open_raw10_next,returns_res10_next,universe,date,label,Open,Close,Volume,returns_close_raw,returns_open_raw,returns_close_raw10,returns_open_raw10,weekday,month,year,day,quarter,macd,sig,squeeze,rsi,Volume_bin_category_0.0,Volume_bin_category_1.0,Volume_bin_category_2.0,Volume_bin_category_3.0,Volume_bin_category_4.0,macd_cross,macd_sharp_rise,macd_div,low_vol,month_name_April,month_name_August,month_name_December,month_name_February,month_name_January,month_name_July,month_name_June,month_name_March,month_name_May,month_name_November,month_name_October,month_name_September,year_obj_2014_,year_obj_2015_,year_obj_2016_,quarter_obj_1_,quarter_obj_2_,quarter_obj_3_,quarter_obj_4_
0,ADBE,-0.018992,0.005233,1.0,20140814,1,0.722705,0.724569,-0.361407,0.025106,0.073793,0.083829,-0.109093,0.708065,0.274483,-1.744504,-0.19357,0.28895,-0.197325,-0.181766,-0.081502,0.307001,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,GOOG,0.011556,0.027871,1.0,20140814,1,2.308370,2.307223,-0.385400,-0.000953,0.103075,0.046072,-0.032728,0.708065,0.274483,-1.744504,-0.19357,0.28895,-0.659924,-0.276354,-0.340211,0.178289,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,DIS,-0.021910,-0.002255,1.0,20140814,0,0.896611,0.901781,0.139298,0.076270,0.022446,0.205148,0.115051,0.708065,0.274483,-1.744504,-0.19357,0.28895,0.192519,0.177279,-1.355040,1.047377,0,0,0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,IGLD,0.067001,0.077727,1.0,20140814,1,-0.740364,-0.731177,-0.457087,0.131876,0.046894,0.297260,0.075415,0.708065,0.274483,-1.744504,-0.19357,0.28895,-0.004539,-0.036059,0.367181,0.592041,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,TRV,-0.032200,-0.017961,0.0,20140814,0,0.924676,0.931889,-0.378999,0.068945,-0.007279,0.192742,0.055753,0.708065,0.274483,-1.744504,-0.19357,0.28895,-0.268020,-0.410394,-0.498566,0.965190,0,0,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [63]:
@pipe
def macdFeats(X):
    
    """
    If macd is negative/positive, the shorter term mv avg is below/above the longer 
    term and momentum is downward/upward - ish.
    
    If the signal is simply the ema of macd. If macd is less/greater than the signal,
    whether pos or neg, the momentum is bear/bullish
    
    Ex
    -----
    both signal and macd are negative. However, macd is noticably less
    negative than the signal. Interpretation: the asset has been in a short term down
    trend (short term down trend could be part of a long term down/up trend). However, 
    the momentum is bullish
    
    Metrics for indicators
    ------
    Crossover: macd is below/above signal
    
    Divergence: the price diverges by a certain threshold from macd. End of 
    current trend
    
    Dramatic Rise: macd is significantly above signal - overbought
       
    """
    
    # compute macd
    asset_close = X.groupby('asset')['Close']
    ema_26 = asset_close.transform(lambda x: x.ewm(span=26).mean()).values
    ema_12 = asset_close.transform(lambda x: x.ewm(span=12).mean()).values
    macd = ema_12 - ema_26
    
    del asset_close, ema_12, ema_26
    
    # Need to define this feature for the groupby
    X['macd'] = macd
    sig =(
        X.groupby('asset')['macd'].
        transform(lambda x: x.ewm(span=9).
        mean()).
        values
        )
    X['sig'] = sig
    
    diff = macd - sig
    close = X['Close'].values
    div = abs(close - macd)
    
    # Define the bools - explain thresholds
    X['macd_cross'] = np.where(macd > sig, 1, 0).astype('bool')
    X['macd_sharp_rise'] = np.where(diff > 0.09, 1, 0).astype('bool')
    X['macd_div'] = np.where(div > 75, 1, 0).astype('bool')
    
    del macd, sig, diff, close, div
    gc.collect()
    
    return X
    
    
    
# macdFeats(data).tail()

In [68]:
@pipe
def bbSqueeze(X):
    
    '''price moves toward upper indicate bullish. This
    returns the distance from stock price to upper or lower 
    BB. If the distance turns negative, the price is above/below. 
    See indicator lambdas.
    
    Squeeze is calculate per asset. However, comparing it to the 6 month min is a bit
    more involved. For now, leave it external. It can be integrated into the func/class
    later if it is useful. Notice the shift on the rolling min(). The present squeeze value
    should not be included in the window we are comparing it with.
    '''
    
    asset_close = X.groupby('asset')['Close']
    
    sd_20 = (
            asset_close.
            transform(lambda x: x.rolling(window=20).
            std()).
            values        
            )
    
    sma_20 = (
            asset_close.
            transform(lambda x: x.rolling(window=20).
            mean()).
            values
            )
    
    # convert everything to arrays   
    close = X['Close'].values   
    U = sma_20 + (sd_20*2)    
    L = sma_20 - (sd_20*2) 
    squeeze = (U - L)/ sma_20
    
    del U, L, asset_close
        
    X['squeeze'] = squeeze
    # We want the nulls - hence min period
    sq_min = (
            X.groupby('asset')['squeeze'].
            transform(lambda x: x.rolling(window=126, min_periods=126).
            min().
            shift())
            )
    # Use to drop null, drop from training features
    X['sq_min'] = sq_min
    
    # Define bool
    X['low_vol'] = np.where(squeeze <= sq_min, 1, 0).astype('bool')

    del squeeze, close, sq_min
    gc.collect()
        
    return X

# bbSqueeze(data).tail()

In [72]:
@pipe
def rsiFeats(X):    
    """
    May consider adding bools or binning
    """
    
    def rsi(x):    
        diff = x.diff()
        mask = diff < 0
        high = abs(diff.mask(mask)).fillna(0)
        low = abs(diff.mask(~mask)).fillna(0)
        pos_rsi = high.ewm(span=14).mean()
        neg_rsi = low.ewm(span=14).mean()
        
        del diff, mask, high, low
        
        return  pos_rsi / (pos_rsi + neg_rsi)
    
    X['rsi'] = (
                X.groupby('asset')['Close'].
                transform(lambda x: rsi(x)).
                values
                )
    
    gc.collect()
    
    return X

# rsiFeats(data).tail()    

In [76]:

# Define bools - this is a bandaid for an issue with Quickpipe
bool_feats = (
        TypeSelector(np.bool_).
        fit_transform(data).
        columns.
        tolist()
        )

transformer_list = [ ('numeric', make_pipeline(TypeSelector(np.number),
                            QuickPipeline_mod()
                            )
                    ),('binned_features', make_pipeline(
                        TypeSelector(np.number),
                        SelectFeatures(feat_list=['Volume']),
                        KBins(n_bins=5),
                        QuickPipeline_mod()
                         )
                    ), ('boolean_features', make_pipeline(
                             TypeSelector(np.bool_),
                            QuickPipeline_mod(categorical_features=bool_feats)
                            )
                   ),('categorical_features', make_pipeline(
                         TypeSelector(np.object),
                        QuickPipeline_mod() )
                        )                                           
                   ]





In [103]:
test.head()

,asset,Open,Close,Volume,returns_close_raw,returns_open_raw,returns_close_raw10,returns_open_raw10,returns_open_raw10_next,returns_res10_next,weekday,month,year,day,quarter,month_name,year_obj,quarter_obj,macd,sig,macd_cross,macd_sharp_rise,macd_div,squeeze,sq_min,low_vol,rsi,universe,date,label
0,QCOM,74.470001,74.720001,10263200.0,0.002814,0.013519,0.018914,0.011615,0.024330,-0.015252,3,1,2014,16,1,January,2014_,1_,0.0,0.0,False,False,False,NaN,NaN,False,NaN,1.0,20140116,0
1,KEP,16.660000,16.719999,205300.0,0.009012,0.000000,0.017497,0.009650,0.025533,-0.010187,3,1,2014,16,1,January,2014_,1_,0.0,0.0,False,False,False,NaN,NaN,False,NaN,1.0,20140116,0
2,NWS,17.059999,17.040001,506000.0,-0.006435,-0.011076,-0.048678,-0.044707,0.087544,0.050676,3,1,2014,16,1,January,2014_,1_,0.0,0.0,False,False,False,NaN,NaN,False,NaN,1.0,20140116,1
3,BHP,66.570000,66.889999,5735900.0,0.029432,0.033296,-0.001494,-0.012391,0.051633,0.013469,3,1,2014,16,1,January,2014_,1_,0.0,0.0,False,False,False,NaN,NaN,False,NaN,1.0,20140116,1
4,NGG,70.043671,69.934494,212800.0,0.001875,0.004530,-0.004361,-0.009308,-0.002802,-0.042414,3,1,2014,16,1,January,2014_,1_,0.0,0.0,False,False,False,NaN,NaN,False,NaN,1.0,20140116,0


In [143]:
@pipe
def preProcessor(X):
    
    global dates, target, label, feats2
    
    
    X.dropna(inplace=True)
    X.drop(['sq_min'], axis=1, inplace=True)
    
    # only use universe for real data simulation
    X['universe'] = universe_feat(X) # imported
    X['date'] = X.index.strftime("%Y%m%d").astype(int)
    X.reset_index(drop=True, inplace=True)
    # Needed for dataframe splits
    # global
    dates = X['date'].unique()
    

    # Convert targets to binary
    # global
    target = ['returns_res10_next']
    label = ['label']   
    X[label[0]] = (X[target[0]] > 0).astype(int)
    
    
    def train_feats(df, drops=list):
        # could do list comp
        train_feat = []
        for col in df.columns:
            if col not in drops:
                train_feat.append(col)
        return train_feat
    
    feats1 = train_feats(X, drops=['date', 'asset', 'universe', 
                               'returns_open_raw10_next', 'returns_res10_next',
                               'label'])
    

    
    transformer_list = [ ('numeric', make_pipeline(TypeSelector(np.number),
                            QuickPipeline_mod()
                            )
                    ),('binned_features', make_pipeline(
                        TypeSelector(np.number),
                        SelectFeatures(feat_list=['Volume']),
                        KBins(n_bins=5),
                        QuickPipeline_mod()
                         )
                    ), ('boolean_features', make_pipeline(
                             TypeSelector(np.bool_),
                            QuickPipeline_mod(categorical_features=(TypeSelector(np.bool_).
                                                                    fit_transform(X).
                                                                    columns.
                                                                    tolist()
                                                                    ))
                            )
                   ),('categorical_features', make_pipeline(
                         TypeSelector(np.object),
                        QuickPipeline_mod() )
                        )                                           
                   ]
    
    preprocess_pipeline = make_pipeline(PandasFeatureUnion(transformer_list))
    
    print('Transforming....')
    X_p = preprocess_pipeline.fit_transform(X[feats1])
    
    X_ = pd.concat([X[[col for col in X.columns if col not in feats1]] ,
                  X_p],
                  axis=1)
    
    # Rerun train features for processed data
    # global
    feats2 = train_feats(X_, drops=['date', 'asset', 'universe', 
                                   'returns_open_raw10_next', 'returns_res10_next',
                                   'label'])
    
    del feats1, X_p, X
    gc.collect()
    
    return X_


    


    
    
# preProc(test2).head()

In [152]:
data = csv.copy()


test =  data >> \
        exTractTime(atts=['weekday', 'month', 'year', 'day', 'quarter'], 
                    mthds=['month_name']) >> \
        toCatFeat(feats=['year', 'quarter']) >> \
        macdFeats() >> \
        bbSqueeze() >> \
        rsiFeats() >> \
        preProcessor()


test.head()

Transforming....


,asset,returns_open_raw10_next,returns_res10_next,universe,date,label,Open,Close,Volume,returns_close_raw,returns_open_raw,returns_close_raw10,returns_open_raw10,weekday,month,year,day,quarter,macd,sig,squeeze,rsi,Volume_bin_category_0.0,Volume_bin_category_1.0,Volume_bin_category_2.0,Volume_bin_category_3.0,Volume_bin_category_4.0,macd_cross,macd_sharp_rise,macd_div,low_vol,month_name_April,month_name_August,month_name_December,month_name_February,month_name_January,month_name_July,month_name_June,month_name_March,month_name_May,month_name_November,month_name_October,month_name_September,year_obj_2014_,year_obj_2015_,year_obj_2016_,quarter_obj_1_,quarter_obj_2_,quarter_obj_3_,quarter_obj_4_
0,ADBE,-0.018992,0.005233,1.0,20140814,1,0.722705,0.724569,-0.361407,0.025106,0.073793,0.083829,-0.109093,0.708065,0.274483,-1.744504,-0.19357,0.28895,-0.197325,-0.181766,-0.081502,0.307001,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,GOOG,0.011556,0.027871,1.0,20140814,1,2.308370,2.307223,-0.385400,-0.000953,0.103075,0.046072,-0.032728,0.708065,0.274483,-1.744504,-0.19357,0.28895,-0.659924,-0.276354,-0.340211,0.178289,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,DIS,-0.021910,-0.002255,0.0,20140814,0,0.896611,0.901781,0.139298,0.076270,0.022446,0.205148,0.115051,0.708065,0.274483,-1.744504,-0.19357,0.28895,0.192519,0.177279,-1.355040,1.047377,0,0,0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,IGLD,0.067001,0.077727,1.0,20140814,1,-0.740364,-0.731177,-0.457087,0.131876,0.046894,0.297260,0.075415,0.708065,0.274483,-1.744504,-0.19357,0.28895,-0.004539,-0.036059,0.367181,0.592041,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,TRV,-0.032200,-0.017961,1.0,20140814,0,0.924676,0.931889,-0.378999,0.068945,-0.007279,0.192742,0.055753,0.708065,0.274483,-1.744504,-0.19357,0.28895,-0.268020,-0.410394,-0.498566,0.965190,0,0,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [158]:
def TrainMyModel(X):
    
    train_range = range(len(dates))[:int(0.90*len(dates))]
    val_range = range(len(dates))[int(0.90*len(dates)):]
    
#     val_range = range(len(dates))[int(0.70*len(dates)):int(0.9*len(dates))]
#     test_range = range(len(dates))[int(0.9*len(dates)):]
    
    
    # train data - numpys
    X_train = X[feats2].loc[X['date'].isin(dates[train_range])].values
    y_train = X[label].loc[X['date'].isin(dates[train_range])].values    
    print(X_train.shape, y_train.shape)
    lgb_train = lgb.Dataset(X_train, y_train[:,0], feature_name=feats2)
    del X_train, y_train
    
    
    # validation data - numpys
    X_val = X[feats2].loc[X['date'].isin(dates[val_range])].values
    y_val = X[label].loc[X['date'].isin(dates[val_range])].values    
    print(X_val.shape, y_val.shape)
    lgb_val = lgb.Dataset(X_val, y_val[:,0], feature_name=feats2),
    del X_val, y_val
    
#     # test data
#     X_test = X[feats2].loc[X['date'].isin(dates[test_range])].values
#     y_test = X[label].loc[X['date'].isin(dates[test_range])].values
#     print(X_test.shape, y_test.shape)
    
    
    param = {"objective" : "binary",
          "metric" : "binary_logloss",
          "verbosity" : -1,
        'random_state': 81}

    model = lgb.train(param, lgb_train, valid_sets=lgb_val, 
                  early_stopping_rounds=10) 
    
    gc.collect()
    return model  #, (X_test, y_test)
                  
    

In [159]:
model = TrainMyModel(test)

(25802, 44) (25802, 1)
(2911, 44) (2911, 1)
[1]	valid_0's binary_logloss: 0.692522
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.691116
[3]	valid_0's binary_logloss: 0.690812
[4]	valid_0's binary_logloss: 0.69059
[5]	valid_0's binary_logloss: 0.691339
[6]	valid_0's binary_logloss: 0.69072
[7]	valid_0's binary_logloss: 0.689664
[8]	valid_0's binary_logloss: 0.689202
[9]	valid_0's binary_logloss: 0.689002
[10]	valid_0's binary_logloss: 0.689558
[11]	valid_0's binary_logloss: 0.688829
[12]	valid_0's binary_logloss: 0.68894
[13]	valid_0's binary_logloss: 0.689471
[14]	valid_0's binary_logloss: 0.688966
[15]	valid_0's binary_logloss: 0.688532
[16]	valid_0's binary_logloss: 0.688716
[17]	valid_0's binary_logloss: 0.689935
[18]	valid_0's binary_logloss: 0.689615
[19]	valid_0's binary_logloss: 0.690175
[20]	valid_0's binary_logloss: 0.689822
[21]	valid_0's binary_logloss: 0.69024
[22]	valid_0's binary_logloss: 0.68942
[23]	valid_0's binary_loglos

In [149]:
# FULL TEST SET DATASET VERSION
preds = model.predict(X_test, num_iteration=model.best_iteration) 
date_vec = df['date'].loc[df['date'].isin(dates[test_range])]
u = df['universe'].loc[df['date'].isin(dates[test_range])]
actual = df[target].fillna(0).loc[df['date'].isin(dates[test_range])].values[:, 0]


custom_metric(date_vec, preds, actual, u)

NameError: name 'X_test' is not defined